# Colour classifying neural network classification regions
This is an interactive Jupyter Notebook for showing the colour classification distributions of my colour classifying neural network.

Importing all the things that we need, also setting the filename for the weights file.

In [1]:
from main import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

FILENAME = "weights.npz"

Importing the weights from the file.

In [2]:
hl_weights, ol_weights = import_weights(FILENAME)

Defining a function that will be useful for the next cell.

In [3]:
# By ChatGPT
def detect_pixel_differences(image):
    # Ensure the input image is a NumPy array
    image = np.array(image)
    
    # Get the shape of the image
    rows, cols = image.shape
    
    # Create an output map initialized with False
    difference_map = np.full((rows, cols), False, dtype=bool)
    
    # Calculate differences with surrounding pixels
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Get the central pixel value
            center_pixel = image[i, j]
            
            # Get the values of surrounding pixels
            surrounding_pixels = image[i - 1:i + 2, j - 1:j + 2]
            
            # Check if any surrounding pixel is different from the center pixel
            if not np.all(surrounding_pixels == center_pixel):
                difference_map[i, j] = True
    
    return difference_map

Creating an interactive window to show a 2D representation of two of the RGB values changing as one is held constant, classifying those colours, and drawing separation lines between different colours. This code is really slow right now (the slowest part is defining the region outlines), I need to find a way to make it faster.

In [4]:
@interact(slider=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5), 
          slider_colour=widgets.RadioButtons(options=['Red', 'Green', 'Blue'],description='Select colour to change with slider:'))
def update_canvas(slider, slider_colour):
    selected_colour = slider_colour
    # Create a canvas with specified dimensions
    canvas_width = 200
    canvas_height = 200

    # Create a canvas with the gradient
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    for y in range(canvas_height):
        for x in range(canvas_width):
            # Interpolate the color at each pixel
            colour1 = x/canvas_width*255
            colour2 = y/canvas_height*255
            colour3 = slider*255
            
            if selected_colour == "Blue":
                canvas[y, x] = [colour1, colour2, colour3]
            elif selected_colour == "Green":
                canvas[y, x] = [colour1, colour3, colour2]
            elif selected_colour == "Red":
                canvas[y, x] = [colour3, colour1, colour2]
                
    classifications = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    for y in range(canvas_height):
        for x in range(canvas_width):
            current_colour = [c/255 for c in canvas[y, x]]
            classifications[y, x] = np.argmax(forward_pass(np.array(current_colour), hl_weights, ol_weights))
        
    difference_map = detect_pixel_differences(classifications)
    rows, cols = difference_map.shape
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            if difference_map[i, j]:
                canvas[i, j] = [255, 255, 255]
                
    centroids = []
    for i in range(0, len(COLOUR_NAMES)):
        x_indices, y_indices = np.where(classifications == i)
        centroid_x = np.mean(x_indices)
        centroid_y = np.mean(y_indices)
        plt.text(centroid_y-10, centroid_x, COLOUR_NAMES[i], fontsize=12, color="black")

    # Display the canvas with the color gradient
    plt.imshow(canvas)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='slider', max=1.0), RadioButtons(description='Select …